# Figure 1

NOTE: If you're running this on **Google Colab**, then uncomment and run the next two cells.

In [ ]:
# !git clone https://github.com/Mark-Kramer/Aperiodic-Exponent-Model.git

In [ ]:
# import sys
# sys.path.insert(0,'/content/Aperiodic-Exponent-Model')

---

## Load packages

In [ ]:
import numpy as np
import scipy.io as io
import matplotlib.pyplot as plt
from   matplotlib.backends.backend_pdf import PdfPages

## Define useful functions

In [ ]:
# Simulate the generative model.
def damped_oscillator_with_noise(noise):
    omega  = 2*np.pi*6          # Fix natural frequency.
    delta  = 1                  # Fix damping coefficient.
    dt     = 0.0001             # Small time step, to deal with noise.
    N      = 100000             # Total number of steps.

    V = np.zeros([N])           # Output variables
    I = np.zeros([N])
    t = np.arange(N)*dt
    
    for n in np.arange(N-1):    # Simulate the model
        V[n+1]  = V[n]  + dt*(I[n])                        + noise["voltage"]*np.random.randn()*np.sqrt(dt)
        I[n+1]  = I[n]  + dt*(-delta*I[n] - omega**2*V[n]) + noise["current"]*np.random.randn()*np.sqrt(dt)
    
    return V,I,t

# Compute the spectrum.
def compute_spectrum(x,t):                         # Compute the spectrum of signal x, time axis t in [s].
    N   = np.size(x)                               # Number of data points
    dt  = t[2]-t[1]                                # Time resolution, in [s].
    T   = t[-1]                                    # Total time of data, in [s].
    xf  = np.fft.fft(np.hanning(N)*(x-np.mean(x))) # Fourier transform of data, Hanning taper, 0-mean.
    S   = np.real(2*dt**2/T*(xf*np.conj(xf)))      # Spectrum
    S   = S[1:int(N/2)+1]                          # Keep only non-negative frequencies
    df  = 1/T                                      # Frequency resolution, in [Hz]
    fNQ = 1/dt/2                                   # Nyquist frequency, in [Hz]
    f   = np.arange(0,fNQ,df)                      # Frequency axis, in [Hz]
    return S, f

# Compute the aperiodic exponent.
def estimate_aperiodic_exponent(S,f,finterval):
    freq_interval_to_fit = (f >= finterval[0]) & (f<=finterval[1])      # For this frequency range,
                                                                        # Fit linear model: log10(S) vs log10(f)
    linear_fit           = np.polyfit(np.log10(f[freq_interval_to_fit]), np.log10(S[freq_interval_to_fit]), 1)
    x_linear_fit         = np.log10(f[freq_interval_to_fit])            # Return x-axis of fit.
    y_linear_fit         = linear_fit[1] + linear_fit[0]*x_linear_fit   # Return y-axis of fit.
    aperiodic_exponent   = linear_fit[0]                                # Return aperiodic exponent.
    return aperiodic_exponent, x_linear_fit, y_linear_fit


## Figure 1A,B: Simulate the model, and plot it.

In [ ]:
noise = {"voltage":[],                          # Set the voltage noise,
         "current":1  }                         # ... and the current noise.
fig, ((ax1, ax2,), (ax3, ax4)) = plt.subplots(2, 2,figsize=(12, 6), dpi=80)

noise["voltage"]=0.0;                           # Voltage noise = 0.
[V,I,t]= damped_oscillator_with_noise(noise)    # Simulate model,
[S,f]  = compute_spectrum(V,t)                  # ... compute the spectrum & fit aperiodic exponent.
[aperiodic_exponent, x_linear_fit, y_linear_fit] = estimate_aperiodic_exponent(S,f,[20,250])

ax1.plot(t,V, 'k');                              ax1.set(xlabel="Time [s]", ylabel='[a.u.]')
ax3.plot(np.log10(f[2:]), np.log10(S[2:]), 'k'); ax3.set(xlabel="Log$_{10}$(Frequency [Hz]", ylabel="Log$_{10}$(P)")
ax3.plot(x_linear_fit, y_linear_fit, 'r');       ax3.text(x_linear_fit[0],y_linear_fit[0]+2, "Aperiodic exponent: %.2f" % aperiodic_exponent);

noise["voltage"]=0.03;                          # Voltage noise > 0.
[V,I,t]= damped_oscillator_with_noise(noise)    # Simulate model,
[S,f]  = compute_spectrum(V,t)                  # ... compute the spectrum & fit aperiodic exponent.
[aperiodic_exponent, x_linear_fit, y_linear_fit] = estimate_aperiodic_exponent(S,f,[20,250])

ax2.plot(t,V, 'k');                              ax2.set(xlabel="Time [s]", ylabel='[a.u.]')
ax4.plot(np.log10(f[2:]), np.log10(S[2:]), 'k'); ax4.set(xlabel="Log$_{10}$(Frequency [Hz]", ylabel="Log$_{10}$(P)")
ax4.plot(x_linear_fit, y_linear_fit, 'r');       ax4.text(x_linear_fit[0],y_linear_fit[0]+2, "Aperiodic exponent: %.2f" % aperiodic_exponent);

                                                # Prettify plots
ax1.spines["top"].set_visible(False); ax1.spines["right"].set_visible(False)
ax2.spines["top"].set_visible(False); ax2.spines["right"].set_visible(False)
ax3.spines["top"].set_visible(False); ax3.spines["right"].set_visible(False)
ax4.spines["top"].set_visible(False); ax4.spines["right"].set_visible(False)

#fig.savefig("./PDFs/Figure-1AB.pdf", bbox_inches='tight')

## Figure 1C (Part 1): Iterative over values of voltage noise and plot aperiodic exponent
This part is slow.

In [ ]:
noise = {"voltage":[],                                     # Prepare the noise variable.
         "current":1  }                                    # ... fix the current noise,
V_noise_values = np.arange(0.0001,0.03, 0.0002)            # ... define range of voltage noises.
N_replicates   = 100                                       # Number of times to repeat simulation.
aperiodic_exponents = np.zeros([np.size(V_noise_values), N_replicates])

for [count, V_noise] in enumerate(V_noise_values):         # For each voltage noise,
    noise["voltage"]=V_noise
    print("V_noise: %.4f" % V_noise)
    for j in np.arange(N_replicates):
        [V,I,t]= damped_oscillator_with_noise(noise)       # Simulate model N_replicate times,
        [S,f]  = compute_spectrum(V,t)                     # ... compute the spectrum,
        ae = estimate_aperiodic_exponent(S,f,[20,250])[0]  # ... fit aperiodic exponent,
        aperiodic_exponents[count, j]   = ae               # ... and save it.
                                                           # Save the results.
res = {"V_noise_values":V_noise_values, "aperiodic_exponents":aperiodic_exponents}
io.savemat(str('Figure-1C.mat'), res)

## Figure 1C (Part 2): Load the aperiodic exponent, and plot it.

This part is fast.

In [ ]:
# Note: if you're running on Google Colab, then load the .mat file like this:
# res = io.loadmat("/content/Aperiodic-Exponent-Model/Figure-1.mat");

# Or, if not running on Google Colab, then load the .mat file like this:
res = io.loadmat("Figure-1C.mat")

aperiodic_exponents = res["aperiodic_exponents"]
V_noise_values      = res["V_noise_values"]

bounds = np.quantile(aperiodic_exponents,[0.025,0.975],1)
mean   = np.mean(aperiodic_exponents,1)

fig = plt.figure(figsize=(12, 4), dpi=80)
plt.plot(np.squeeze(V_noise_values),mean,'k')
plt.plot(np.squeeze(V_noise_values),bounds[0,:],'red')
plt.plot(np.squeeze(V_noise_values),bounds[1,:],'red')
plt.fill_between(np.squeeze(V_noise_values),bounds[0,:],bounds[1,:], facecolor='red', alpha=0.5)
plt.grid(True); plt.xlim([0.0, 0.03])
plt.xlabel('Voltage Noise'); plt.ylabel('Aperiodic Exponent');

#fig.savefig("./PDFs/Figure-1C.pdf", bbox_inches='tight')